# imports

In [ ]:
from fastai.vision.all import *

# functions

## 应对无目标的情况

In [ ]:
def bb_pad_intlbl(samples, pad_idx=0):
    "Function that collect `samples` of labelled bboxes and adds padding with `pad_idx`."
    samples = [(s[0], *clip_remove_empty(*s[1:])) for s in samples]
    max_len = max([len(s[2]) for s in samples])
    def _f(img,bbox,lbl):
        bbox = torch.cat([bbox,bbox.new_zeros(max_len-bbox.shape[0], 4)])
        #lbl  = torch.cat([lbl, lbl .new_zeros(max_len-lbl .shape[0], int)+pad_idx])
        #在无目标也就是lbl为[]的情况下，lbl  = torch.cat([lbl, lbl .new_zeros(max_len-lbl .shape[0])+pad_idx])
        #上面的代码即使指定了dtype=torch.int得到的仍然是浮点数。会导致后面的报错不是索引
        if lbl.shape[0] != 0:
            lbl  = torch.cat([lbl, lbl .new_zeros(max_len-lbl .shape[0])+pad_idx])
        else:
            lbl = lbl.new_zeros(max_len, dtype = torch.int) + pad_idx

        
        return img,bbox,lbl
    return [_f(*s) for s in samples]

BBoxBlock = TransformBlock(type_tfms=TensorBBox.create, item_tfms=PointScaler, dls_kwargs = {'before_batch': bb_pad_intlbl})

## 获取BBox和label  
两个是分开进行的。并且BBox的顺序改成了先x后y，使用v1版的fastai的数据集的时候需要转换顺序。

In [ ]:
#export
pat_coord = re.compile(r'\d+')
pat_clas = re.compile(r'\w+')
pat_imgName = re.compile(r'(\w+/\d+\.png)$')
pat_imgName = re.compile(r'(\w+/\d+\.jpg)$')
def get_label_from_df(fn, df, pat_imgName, box_col, cat_col):    
    fn = str(fn)
    pat_cat = re.compile(r'\w+')
    
    fn = pat_imgName.findall(str(fn))[0]
    cats = df.loc[fn,cat_col]
    cats = pat_clas.findall(cats)
    
    return cats

def get_boxes_from_df(fn, df, pat_imgName, box_col, cat_col):
    fn = str(fn)
    pat_num = re.compile(r'\d+')
    pat_cat = re.compile(r'\w+')
    fn = pat_imgName.findall(str(fn))[0]
    #print('dbg1', fn)
    
    boxes = df.loc[fn,box_col]
    boxes = pat_num.findall(boxes)
    #boxes = list(map(np.long, boxes))
    boxes = list(map(np.int32, boxes))
    boxes = np.array(boxes).reshape(-1,4)
    
    #fastai2里面bbox的顺序改成了xy的顺序。现在用的这个数据集还是v1里面的yx的顺序。这里调整一下
    boxes = boxes[...,[1, 0, 3, 2]]
    boxes = boxes.tolist()
    
    cats = df.loc[fn,cat_col]
    cats = pat_clas.findall(cats)
    #print('dbg2', fn, boxes, cats)
    assert len(boxes)==len(cats), 'length of bounding boxes and categories not equeal.'
    
    #print('dbg_boxes:', boxes)    
    return boxes

# test

In [ ]:
src_path = '/home/dev/jupyter/detect_symbol/data/ds_20200429/'

In [ ]:
path = src_path + 'images'

In [ ]:
path = Path(path)

In [ ]:
path.ls()

In [ ]:
df = pd.read_csv(src_path + 'gends.csv',index_col=0)
df = df.set_index('image')
df.head()

In [ ]:
get_y1 = partial(get_boxes_from_df, df=df, pat_imgName=pat_imgName, box_col='box', cat_col='cls')
get_y2 = partial(get_label_from_df, df=df, pat_imgName=pat_imgName, box_col='box', cat_col='cls')
#print(BBoxBlock.dls_kwargs)
syms = DataBlock(blocks=(ImageBlock, BBoxBlock, BBoxLblBlock),
                 get_items=get_image_files,
                 splitter=RandomSplitter(),
                 get_y=[get_y1, get_y2],
                 #item_tfms=Resize(128),
                 #batch_tfms=aug_transforms(),
                 n_inp=1)

In [ ]:
dls = syms.dataloaders(path)

In [ ]:
dls.show_batch(max_n = 1)

In [ ]:
#syms.summary(path)

In [ ]:
#dts = syms.datasets(path)

In [ ]:
#dts[1]

In [ ]:
#df.loc['images/02364.jpg']

In [ ]:
def dbg():
    import pdb;pdb.set_trace()
    dls.show_batch()
    #dls = syms.dataloaders(path)
dbg()

In [ ]:
coco_source = untar_data(URLs.COCO_TINY)

In [ ]:
images, lbl_bbox = get_annotations(coco_source/'train.json')
img2bbox = dict(zip(images, lbl_bbox))

In [ ]:
coco_source

In [ ]:
def get_y1(o):
    #print('dbg0', o)
    #return [img2bbox[o.name][0], img2bbox[o.name][1]]
    return img2bbox[o.name][0]

def get_y2(o): 
    return img2bbox[o.name][1]

coco = DataBlock(blocks=(ImageBlock, BBoxBlock, BBoxLblBlock),
                 get_items=get_image_files,
                 splitter=RandomSplitter(),
                 get_y=[get_y1, get_y2], #[lambda o: img2bbox[o.name][0], lambda o: img2bbox[o.name][1]]
                 item_tfms=Resize(128),
                 batch_tfms=aug_transforms(),
                 n_inp=1)

In [ ]:
cocodls = coco.dataloaders('/root/.fastai/data/coco_tiny')

In [ ]:
cocodls.show_batch()


In [ ]:
# pets = DataBlock(blocks = (ImageBlock, CategoryBlock),
#                  get_items=get_image_files, 
#                  splitter=RandomSplitter(seed=42),
#                  get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
#                  item_tfms=Resize(460),
#                  batch_tfms=aug_transforms(size=224, min_scale=0.75))
# dls = pets.dataloaders(path/"images")